# Pytorch Tabular
Notebook pensando para facilitar e agiliar o treinamento de Deep Learning, sendo necessário, em grande parte das vezes, somente alterar o caminho do dataset e o tipo (classificador ou regressão). Assim como, quais arquiteturas serão utilizado para buscar os melhores hiperparâmetros.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import time
import math
import json

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Pytorch Tabular
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

## Preparar Dataset

In [2]:
def preprocess_data(df, target_cols, max_unique_values=10, window_size=1):
    """
    Preprocess dataset, converting string columns to number and identifying numeric, categorical, and date columns.
    Adds a window of historical rows to the data.

    Args:
        df (DataFrame): DataFrame with data to analyze.
        target_cols (list): All target columns to not add in numeric, categorical, or date feature lists.
        max_unique_values (int): Maximum number of unique values to consider a numeric column as categorical.
        window_size (int): Number of previous rows to include for each row.

    Returns:
        df_copy (DataFrame): Processed DataFrame with transformations applied.
        num_col_names (list): List of numeric feature column names.
        cat_col_names (list): List of categorical feature column names.
        date_col_names (list): List of date feature column names.
        mappings (dict): Mapping of original categorical values (or target columns) to transformed numeric values.
    """
    # Create a copy so as not to alter the original DataFrame
    df_copy = df.copy()

    # Remove lines with null values
    df_copy = df_copy.dropna()
    
    # Identify categorical and numeric columns
    cat_col_names = df_copy.select_dtypes(include=['object', 'category']).columns.tolist()
    num_col_names = df_copy.select_dtypes(include=['number']).columns.tolist()
    date_col_names = []

    # Identify columns that are dates
    for col in df.columns:
        if df[col].dtype in ['object', 'string']:  # Only check object/string columns
            try:
                # Attempt to convert the column to datetime
                pd.to_datetime(df[col], errors='raise')
                if col not in target_cols:  # Avoid considering target columns as date columns
                    date_col_names.append(col)
            except (ValueError, TypeError):
                pass

    # Identify numeric columns that are categorical
    potential_categorical = []
    for col in num_col_names:
        if col not in target_cols and df_copy[col].nunique() <= max_unique_values:
            potential_categorical.append(col)

    cat_col_names += potential_categorical

    # Remove target columns from the lists
    num_col_names = [col for col in num_col_names if col not in potential_categorical + target_cols]
    cat_col_names = [col for col in cat_col_names if col not in target_cols]
    
    mappings = {}
    label_encoders = {}

    # Convert string columns to category and create a mapping (old value -> new value)
    for col in cat_col_names + target_cols:
        if df_copy[col].dtype not in ['int64', 'float64']:
            le = LabelEncoder()
            df_copy[col] = le.fit_transform(df_copy[col])
            mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
            label_encoders[col] = le
            # Convert values to int (otherwise will raise error if save as json)
            for key, value in mappings[col].items():
                mappings[col][key] = int(value)
        #else:
        #    mappings[col] = {int(val): int(val) for val in df_copy[col].unique()}

    # Add historical data based on window_size
    if window_size > 1:
        historical_features = []
        for i in range(window_size - 1, 0, -1):  # Criar features lag, do mais antigo ao mais recente
            shifted = df_copy.drop(columns=target_cols, errors='ignore').shift(i).add_suffix(f'_lag_{i}')
            historical_features.append(shifted)
        
        # Concatenate the lags to the left and keep the current values
        df_copy = pd.concat(historical_features + [df_copy], axis=1)
    
        # Remove NaNs columns created by shifts
        df_copy = df_copy.dropna()
    
        # Add lags
        num_col_lags = [f'{col}_lag_{i}' for i in range(window_size - 1, 0, -1) for col in num_col_names]
        cat_col_lags = [f'{col}_lag_{i}' for i in range(window_size - 1, 0, -1) for col in cat_col_names]
        date_col_lags = [f'{col}_lag_{i}' for i in range(window_size - 1, 0, -1) for col in date_col_names]
        mappings_lags = {}
        for col, value in mappings.items():
            for i in range(window_size - 1, 0, -1):
                mappings_lags[f'{col}_lag_{i}'] = value
    
        # Update main columns + lags columns
        num_col_names = num_col_lags + num_col_names
        cat_col_names = cat_col_lags + cat_col_names
        date_col_names = date_col_lags + date_col_names
        mappings = {**mappings_lags, **mappings}
    
    return df_copy, num_col_names, cat_col_names, date_col_names, mappings

In [3]:
df = pd.read_csv('dataset/synthetic_dataset_cat_1000.csv')
model_type = 'classifier'  # classifier or regressor
window_size = 1
df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,target
0,16.647436,E,183.980936,24.617366,5.717618,-0.631023,1.20,0.866846,type_3
1,10.604436,A,85.459194,307.449148,6.655153,0.767629,1.10,0.977042,type_4
2,77.388714,E,40.558116,155.780617,6.921389,0.015533,1.20,0.803368,type_3
3,90.406079,A,15.880388,33.010214,6.250944,-1.473239,1.10,0.964277,type_4
4,87.356717,E,170.070489,178.530697,6.204453,1.044204,1.20,0.052600,type_4
...,...,...,...,...,...,...,...,...,...
1995,52.551128,B,12.801325,246.448995,5.907150,0.017774,0.90,0.771878,type_2
1996,59.145417,D,156.294131,216.923304,5.872186,0.194822,0.95,0.143240,type_3
1997,22.532483,B,115.050490,217.424978,6.890049,0.120550,0.90,0.550371,type_2
1998,18.649490,F,38.755850,314.397756,6.590435,-0.790071,0.85,1.441102,type_5


In [4]:
target_cols = ['target']
# Aplicar o pré-processamento
df, num_col_names, cat_col_names, date_col_names, category_mappings = preprocess_data(df, target_cols, max_unique_values=10, window_size=window_size)
col_names_order = df.columns.tolist()

In [5]:
print(f'''col_names_order: {col_names_order}
num_col_names: {num_col_names}
cat_col_names: {cat_col_names}
date_col_names: {date_col_names}
target_cols: {target_cols}
category_mappings: {category_mappings}''')

col_names_order: ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'target']
num_col_names: ['feature_1', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_8']
cat_col_names: ['feature_2', 'feature_7']
date_col_names: []
target_cols: ['target']
category_mappings: {'feature_2': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}, 'target': {'type_1': 0, 'type_2': 1, 'type_3': 2, 'type_4': 3, 'type_5': 4}}


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1999 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature_1  1999 non-null   float64
 1   feature_2  1999 non-null   int64  
 2   feature_3  1999 non-null   float64
 3   feature_4  1999 non-null   float64
 4   feature_5  1999 non-null   float64
 5   feature_6  1999 non-null   float64
 6   feature_7  1999 non-null   float64
 7   feature_8  1999 non-null   float64
 8   target     1999 non-null   int64  
dtypes: float64(7), int64(2)
memory usage: 156.2 KB


In [7]:
if model_type == 'classifier':
    print(df[target_cols].value_counts())

target
0         400
1         400
2         400
3         400
4         399
Name: count, dtype: int64


### Dataset Splitting

In [8]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, random_state=42, test_size=0.3)
#valid, test = train_test_split(valid, random_state=42, test_size=0.5)
test = valid

## Train Pytorch Tabular
https://github.com/manujosephv/pytorch_tabular

### Preparar Configurações

No [DataConfig](https://pytorch-tabular.readthedocs.io/en/latest/data/#pytorch_tabular.config.DataConfig) estão as configurações do dataset, contendo as coluna no DataFrame Pandas que é o "target" (valor que será previsto), colunas que possuem dados continuos e de dados categóricos. Além de outras configurações que podem ser alteradas.

In [9]:
data_config = DataConfig(
    target=target_cols,  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,  # Column names of the numeric fields
    categorical_cols=cat_col_names,  # Column names of the categorical fields
    date_columns=date_col_names,  # (Column name, Freq, Format) tuples of the date fields. E.g. [("date", "M", %Y-%m')]. freq = https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
    normalize_continuous_features=True, # Flag to normalize the input features (continuous). Default True
    num_workers=4
)

No [TrainerConfig](https://pytorch-tabular.readthedocs.io/en/latest/training/#using-the-entire-pytorch-lightning-trainer) estão as configurações de treinamento, incluindo quais métricas iremos utilizar para avaliar o modelo e criar os checkpoints

No [OptimizerConfig](https://pytorch-tabular.readthedocs.io/en/latest/optimizer/#pytorch_tabular.config.OptimizerConfig) estão as configurações do otimizado que será utilizador, sendo inclusive possível alterar seus parâmetros.

In [10]:
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate. Default False
    batch_size=64,
    max_epochs=300,
    min_epochs=50,
    early_stopping='valid_accuracy' if model_type == 'classifier' else 'valid_loss', # Monitor for early stopping. valid_loss or valid_accuracy
    early_stopping_mode = 'max' if model_type == 'classifier' else 'min', # Set the mode as min for val_loss (lower is better) and max for val_accuracy (higher is better)
    early_stopping_patience=20, # No. of epochs of degradation training will wait before terminating
    checkpoints='valid_accuracy' if model_type == 'classifier' else 'valid_loss', # Save best checkpoint monitoring
    checkpoints_mode='max' if model_type == 'classifier' else 'min',  # Set the mode as min for valid_loss (lower is better) and max for valid_accuracy (higher is better)
)

optimizer_config = OptimizerConfig(
    #optimizer='RMSprop',  # https://pytorch.org/docs/stable/optim.html#algorithms
    #optimizer_params={'alpha': 0.99},
    #lr_scheduler='StepLR',
    #lr_scheduler_params={'step_size': 10},
)

## Train Many Models
Entre tantas alternativas é difícil saber qual a melhora arquitetura utilizar, sem falar que muitas vezes uma arquitetura tem um desempenho muito bom em determinado assunto/dataset, mas isso não se mostra verdade entre outro cenário, dificultando escolher qual utilizar.

Para ajudar nesse dilema o Pytorch Tabular possui a função [model_sweep](https://pytorch-tabular.readthedocs.io/en/latest/apidocs_coreclasses/#pytorch_tabular.model_sweep) que roda todas as arquiteturas com os hiperpâmetros padrões e seu resultado consiste no comparativo entre todos eles. Dessa forma sabemos em qual/quais arquiteturas devemos investir mais tempo para buscar melhorar seus hiperparâmetros.

Vamos rodar a categoria "high_memory" que irá treinar todos modelos, inclusive os que utilizam bastante memória. Mas não se preocupe, se a memória não for suficiente a biblioteca irá ignorar o modelo e na tabela de resultados irá informar que não tinha memória suficiente (OOM). Ressaltando que pode ser interessante ter um batch_size pequeno para conseguir treinar todos os modelos sem resultar em OOM, ou rodar o modelo específico que deu OOM depois isoladamente com o batch_size que for suficiente :).

In [11]:
from pytorch_tabular import model_sweep, tabular_model_sweep

In [12]:
#tabular_model_sweep.MODEL_SWEEP_PRESETS

In [13]:
tic = time.time()
sweep_df, best_model = model_sweep(
    task='classification' if model_type == 'classifier' else 'regression',  # 'classification' or 'regression'
    train=train,
    test=valid,
    data_config=data_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    model_list='high_memory',  # The list of models to compare. This can be one of the presets defined in pytorch_tabular.tabular_model_sweep.MODEL_SWEEP_PRESETS or a list of ModelConfig objects. Defaults to "lite".
    verbose=True
)
print(f'Total time: {time.time() - tic}')

Output()

2025-02-26 09:33:16,365 - {pytorch_tabular:328} - INFO - Training AutoIntModel

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

2025-02-26 09:33:28,631 - {pytorch_tabular:382} - INFO - Finished Training AutoIntModel

2025-02-26 09:33:28,632 - {pytorch_tabular:383} - INFO - Results: model: AutoIntModel, # Params: 15 T, epochs: 49, 
test_loss_0: 11.459814071655273, test_loss: 11.459814071655273, test_accuracy: 0.2516666650772095, time_taken:     
12.26884150505066, time_taken_per_epoch: 0.25038452051123794

...

2025-02-26 09:38:30,472 - {pytorch_tabular:382} - INFO - Finished Training TabTransformerModel

2025-02-26 09:38:30,473 - {pytorch_tabular:383} - INFO - Results: model: TabTransformerModel, # Params: 272 T,     
epochs: 49, test_loss_0: 0.975267767906189, test_loss: 0.975267767906189, test_accuracy: 0.5983333587646484,       
time_taken: 20.949220418930054, time_taken_per_epoch: 0.4275351105904093

2025-02-26 09:38:30,475 - {pytorch_tabular:388} - INFO - Model Sweep Finished

2025-02-26 09:38:30,476 - {pytorch_tabular:389} - INFO - Best Model: FTTransformerModel

Total time: 314.16385865211487


O melhor checkpoint de todos treinamentos será armazenado na variável de output, que neste caso estamos chamando de "best_model".

In [14]:
test_metric = best_model.evaluate(test)
if model_type == 'regressor':
    print('Test MSE: %.3f' % (test_metric[0]['test_mean_squared_error']))
    print(f'Test inference error (RMSE): ±{math.sqrt(test_metric[0]["test_mean_squared_error"])}')

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7416666746139526     │
│         test_loss         │    0.6228945851325989     │
│        test_loss_0        │    0.6228945851325989     │
└───────────────────────────┴───────────────────────────┘

Lembrando que: Cada vez que rodar o treinamento o resultado pode ser diferente!

In [15]:
# Replace "OOM" with 0.0
sweep_df.replace(['OOM', np.nan], 0.0, inplace=True)
sweep_df.replace([np.inf], 999., inplace=True)

if model_type == 'regressor':
    sweep_df['test_rmse'] = np.sqrt(sweep_df['test_mean_squared_error'])

sweep_df.drop(columns=['params', 'time_taken']).sort_values(
    'test_accuracy' if model_type == 'classifier' else 'test_loss', ascending=False if model_type == 'classifier' else True).style.background_gradient(
    subset=['test_accuracy' if model_type == 'classifier' else 'test_loss'], cmap='RdYlGn'
).background_gradient(subset=['time_taken_per_epoch', 'test_loss'], cmap='RdYlGn_r')

,model,# Params,epochs,test_loss_0,test_loss,test_accuracy,time_taken_per_epoch
3,FTTransformerModel,272 T,49,0.622895,0.622895,0.741667,0.422960
2,DANetModel,426 T,49,0.681066,0.681066,0.713333,0.925443
1,CategoryEmbeddingModel,12 T,43,0.823135,0.823135,0.711667,0.246907
5,GatedAdditiveTreeEnsembleModel,77 T,74,0.884190,0.884190,0.650000,1.578803
6,NODEModel,1 M,49,0.881665,0.881665,0.621667,0.948116
8,TabTransformerModel,272 T,49,0.975268,0.975268,0.598333,0.427535
4,GANDALFModel,7 T,49,1.224026,1.224026,0.451667,0.373522
7,TabNetModel,6 T,49,1.427356,1.427356,0.380000,0.460093
0,AutoIntModel,15 T,49,11.459814,11.459814,0.251667,0.250385


In [16]:
# Save best model
best_model.save_model('results/best_model_pytorch_tabular', inference_only=True)

# Save columns names and informations
data_to_save = {
    'col_names_order': col_names_order,
    'num_col_names': num_col_names,
    'cat_col_names': cat_col_names,
    'date_col_names': date_col_names,
    'target_cols': target_cols,
    'category_mappings': category_mappings,
    'window_size': window_size
}
with open('results/columns_metadata_pytorch_tabular.json', 'w') as json_file:
    json.dump(data_to_save, json_file, indent=4)

2025-02-26 09:38:30,696 - {pytorch_tabular.tabular_model:1575} - WARNING - Directory is not empty. Overwriting the 
contents.

### Fine-tuning Hyperparametrs
Show, agora que sabemos as melhores arquiteturas para o nosso modelo, podemos brincar com ela(s) para buscar melhores hiperparâmetros para termos resultados ainda melhores. Para isso o Pytorch Tabular disponibilizar a função [tuner](https://pytorch-tabular.readthedocs.io/en/latest/apidocs_coreclasses/#pytorch_tabular.TabularModelTuner.tune).

Configurações dos modelos: https://pytorch-tabular.readthedocs.io/en/latest/apidocs_model/

Mais informações de como o hyperparameter space funcionam: https://pytorch-tabular.readthedocs.io/en/latest/tutorials/10-Hyperparameter%20Tuning/#define-the-hyperparameter-space

Neste caso vamos pegar as duas arquiteturas que se sairam bem e brinca com elas.

In [17]:
from pytorch_tabular.tabular_model_tuner import TabularModelTuner

from pytorch_tabular.models import (
    FTTransformerConfig,
    DANetConfig,
)   

As configurações dos spaces que serão utilizados sempre irão começar com nome da configuração, dois underlines e o parâmetro. Sendo seus valores uma lista com todas as configurações que deseja testar.

In [18]:
model_config_FTTransformerConfig = FTTransformerConfig(task='classification' if model_type == 'classifier' else 'regression')

search_space_FTTransformerConfig = {
    'model_config__input_embed_dim': [16, 32, 64],
    'model_config__num_heads': [4, 8, 16],
    'model_config__transformer_activation': ['GEGLU', 'ReLU', 'LeakyReLU'],
}

In [19]:
model_config_DANetConfig = DANetConfig(task='classification' if model_type == 'classifier' else 'regression')

search_space_DANetConfig = {
    'optimizer_config__optimizer': ['Adam', 'AdamW'],
    'model_config__n_layers': [4, 8, 16],
    'model_config__abstlay_dim_1': [16, 32, 64],
    'model_config__abstlay_dim_2': [32, 64, 128],
}

Para informar o tuner quais arquiteturas e hiperparâmetros testar, é necessário adicioná-los em listas. Ressaltando que ambas listas devem estar ordenadas iguais e ter o mesmo tamanho (irá utilizar primeiro elemento de models com primeiro de search_space, segundo com segundo,...)

In [20]:
all_models = [model_config_FTTransformerConfig, model_config_DANetConfig]
all_search_space = [search_space_FTTransformerConfig, search_space_DANetConfig]

O tuner possui duas "strategy" principais:
- grid_search: Para pesquisar todos os hiperparâmetros que foram definidos, mas lembre-se que cada novo campo que você adicionar aumentará consideravelmente o tempo total de treinamento. Se você configurar 4 otimizadores, 4 layes, 2 ativações e 2 dropout, isso significa 64 (4 * 4 * 2 * 2) treinamentos.
- random_search: Obterá aleatoriamente configurações de hiperparâmetros "n_trials" de cada modelo que foi definido. É útil para um treinamento mais rápido, mas lembre-se de que não testará todos os hiperparâmetros.

In [21]:
tuner = TabularModelTuner(
    data_config=data_config,
    model_config=all_models,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

tic = time.time()
tuner_df = tuner.tune(
    train=train,
    validation=valid,
    search_space=all_search_space,
    strategy='grid_search',  # grid_search (seach in all search_spaces) or random_search (search in n_trials random search_spaces)
    n_trials=4,  # Used when strategy is random_search
    metric='accuracy' if model_type == 'classifier' else 'loss',
    mode='max' if model_type == 'classifier' else 'min',  # max or min
    progress_bar=True,
    verbose=True # Make True if you want to log metrics and params each iteration
)
print(f'Total time: {time.time() - tic}')

Output()

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

2025-02-26 09:48:44,357 - {pytorch_tabular.tabular_model_tuner:412} - INFO - Trial 1/27: {'model':                 
'0-FTTransformerConfig', 'model_config__input_embed_dim': 16, 'model_config__num_heads': 4,                        
'model_config__transformer_activation': 'GEGLU', 'loss_0': 0.6435269117355347, 'loss': 0.6435269117355347,         
'accuracy': 0.7549999952316284, 'trial_id': 0} | Score: 0.7549999952316284

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

2025-02-26 09:49:19,986 - {pytorch_tabular.tabular_model_tuner:412} - INFO - Trial 2/27: {'model':                 
'0-FTTransformerConfig', 'model_config__input_embed_dim': 16, 'model_config__num_heads': 4,                        
'model_config__transformer_activation': 'ReLU', 'loss_0': 0.5075352787971497, 'loss': 0.5075352787971497,          
'accuracy': 0.8083333373069763, 'trial_id': 1} | Score: 0.8083333373069763

...

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

2025-02-26 11:02:14,739 - {pytorch_tabular.tabular_model_tuner:412} - INFO - Trial 54/54: {'model':                
'1-DANetConfig', 'model_config__abstlay_dim_1': 64, 'model_config__abstlay_dim_2': 128, 'model_config__n_layers':  
16, 'optimizer_config__optimizer': 'AdamW', 'loss_0': 0.5869177579879761, 'loss': 0.5869177579879761, 'accuracy':  
0.778333306312561, 'trial_id': 53} | Score: 0.778333306312561

2025-02-26 11:02:14,746 - {pytorch_tabular.tabular_model_tuner:427} - INFO - Model Tuner Finished

2025-02-26 11:02:14,747 - {pytorch_tabular.tabular_model_tuner:428} - INFO - Best Model: 1-DANetConfig - Best Score
(accuracy): 0.8199999928474426

Total time: 4435.764271974564


O Tuner retorna uma única variável contendo [tabela deresultados, parâmetros do melhor modelo, valor da melhor métrica, melhor modelo].

In [22]:
if model_type == 'regressor':
    tuner_df.trials_df['test_rmse'] = np.sqrt(tuner_df.trials_df['mean_squared_error'])

tuner_df.trials_df.sort_values('accuracy' if model_type == 'classifier' else 'loss', ascending=False if model_type == 'classifier' else True).style.background_gradient(
    subset=['accuracy' if model_type == 'classifier' else 'loss'], cmap='RdYlGn'
).background_gradient(subset=['loss'], cmap='RdYlGn_r')

,trial_id,model,model_config__input_embed_dim,model_config__num_heads,model_config__transformer_activation,loss_0,loss,accuracy,model_config__abstlay_dim_1,model_config__abstlay_dim_2,model_config__n_layers,optimizer_config__optimizer
46,19,1-DANetConfig,nan,nan,nan,0.512986,0.512986,0.820000,32.000000,32.000000,4.000000,AdamW
45,18,1-DANetConfig,nan,nan,nan,0.552606,0.552606,0.808333,32.000000,32.000000,4.000000,Adam
1,1,0-FTTransformerConfig,16.000000,4.000000,ReLU,0.507535,0.507535,0.808333,nan,nan,nan,nan
55,28,1-DANetConfig,nan,nan,nan,0.572783,0.572783,0.803333,32.000000,64.000000,16.000000,Adam
63,36,1-DANetConfig,nan,nan,nan,0.510869,0.510869,0.801667,64.000000,32.000000,4.000000,Adam
7,7,0-FTTransformerConfig,16.000000,16.000000,ReLU,0.571022,0.571022,0.800000,nan,nan,nan,nan
25,25,0-FTTransformerConfig,64.000000,16.000000,ReLU,0.552217,0.552217,0.795000,nan,nan,nan,nan
64,37,1-DANetConfig,nan,nan,nan,0.557062,0.557062,0.793333,64.000000,32.000000,4.000000,AdamW
27,0,1-DANetConfig,nan,nan,nan,0.592444,0.592444,0.793333,16.000000,32.000000,4.000000,Adam
17,17,0-FTTransformerConfig,32.000000,16.000000,LeakyReLU,0.519083,0.519083,0.793333,nan,nan,nan,nan


In [23]:
tuner_df.best_model.evaluate(test)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8199999928474426     │
│         test_loss         │    0.5129860639572144     │
│        test_loss_0        │    0.5129860639572144     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_0': 0.5129860639572144,
  'test_loss': 0.5129860639572144,
  'test_accuracy': 0.8199999928474426}]

### Save Model

In [24]:
tuner_df.best_model.save_model('results/best_model_tuner_pytorch_tabular', inference_only=True)

# Save columns names and informations
data_to_save = {
    'col_names_order': col_names_order,
    'num_col_names': num_col_names,
    'cat_col_names': cat_col_names,
    'date_col_names': date_col_names,
    'target_cols': target_cols,
    'category_mappings': category_mappings,
    'window_size': window_size
}
with open('results/columns_metadata_tuner_pytorch_tabular.json', 'w') as json_file:
    json.dump(data_to_save, json_file, indent=4)

2025-02-26 11:02:14,973 - {pytorch_tabular.tabular_model:1575} - WARNING - Directory is not empty. Overwriting the 
contents.

## Predict
Precisar estar no formato DataFrame do Pandas com os nomes das colunas que participaram do treinamento para passar para o modelo fazer inferência.

In [25]:
import time
import json
import pickle

import pandas as pd
from pytorch_tabular import TabularModel

In [27]:
loaded_model = TabularModel.load_model('results/best_model_tuner_pytorch_tabular')

# columns_metadata
columns_metadata = json.load(open('results/columns_metadata_tuner_pytorch_tabular.json', 'r'))

2025-02-26 11:03:26,855 - {pytorch_tabular.tabular_model:171} - INFO - Experiment Tracking is turned off

2025-02-26 11:03:26,858 - {pytorch_tabular.tabular_model:342} - INFO - Preparing the Trainer

In [28]:
def preprocess_inference_data(new_input, columns_metadata, scaler=None):
    """
    Preprocess the input data for inference based on metadata and scaler for numeric normalization.

    Args:
        new_input (list of list): Data to preprocess (each inner list is a row).
        columns_metadata (dict): Metadata defining column names, types, mappings, and order.
        scaler (StandardScaler): Trained scaler for numerical columns.
    
    Returns:
        list: Preprocessed data ready for inference.
    """
    # Exclude target columns from col_names_order for inference
    target_cols = columns_metadata['target_cols']
    col_names_order = [col for col in columns_metadata['col_names_order'] if col not in target_cols]
    # Transform the new_input into a DataFrame with the correct column order
    df_input = pd.DataFrame(new_input, columns=col_names_order)

    # Convert categorical columns based on category_mappings
    category_mappings = columns_metadata['category_mappings']
    for col in columns_metadata['cat_col_names']:
        if col in df_input.columns and col in category_mappings:
            # Replace string categories with mapped integer IDs
            df_input[col] = df_input[col].map(category_mappings[col])
            if df_input[col].isnull().any():
                raise ValueError(f'Invalid value found in column "{col}" that is not in the category mappings.')

    # Normalize numeric columns using the trained scaler
    num_col_names = columns_metadata['num_col_names']
    if num_col_names and scaler:
        df_input[num_col_names] = scaler.transform(df_input[num_col_names])

    # Ensure date columns are in the correct datetime format
    for col in columns_metadata['date_col_names']:
        if col in df_input.columns:
            df_input[col] = pd.to_datetime(df_input[col], errors='coerce')
            if df_input[col].isnull().any():
                raise ValueError(f'Invalid date value found in column "{col}".')

    # Drop target columns if they exist in the input (not used in inference)
    target_cols = columns_metadata['target_cols']
    df_input = df_input.drop(columns=target_cols, errors='ignore')

    # Convert DataFrame to list of rows for model inference
    #processed_data = df_input.values.tolist()

    return df_input

In [29]:
new_input = [
    [66.770774, 'A', 129.519443, 157.649826, 6.255495, 0.828523, 1.10, 0.656228]  # target type_4
]
new_input_preprocessed = preprocess_inference_data(new_input, columns_metadata)
new_input_preprocessed

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8
0,66.770774,0,129.519443,157.649826,6.255495,0.828523,1.1,0.656228


In [30]:
target = columns_metadata['target_cols'][0]
clss_to_cat = {}
if target in columns_metadata['category_mappings']:
    for key, value in columns_metadata['category_mappings'][target].items():
        clss_to_cat[value] = key

In [31]:
tic = time.time()
result = loaded_model.predict(new_input_preprocessed)
if clss_to_cat:
    result = clss_to_cat[result['target_prediction'][0]]
print(result)
print(f'Total time: {time.time() - tic}')

type_4
Total time: 0.12233686447143555
